# Tutorial for Pipefunc Package

The `pipefunc` package is a Python library that allows you to define functions as pipelines, with each function providing a single step in the pipeline. In this tutorial, we will explain how to use the package, based on an example notebook.


For the latest documentation, check out [the official documentation](https://pipefunc.readthedocs.io/en/latest/#what-is-this).

---

## Building a Simple Pipeline

Let's start by importing `pipefunc` and `Pipeline` from the `pipefunc` module.


In [ ]:
from pipefunc import pipefunc, Pipeline

We then define some functions using the `@pipefunc` decorator. The `@pipefunc` decorator turns these functions into pipeline steps. For each function, we specify an `output_name` which will be used to refer to the output of that function in the pipeline.


In [ ]:
@pipefunc(output_name="c")
def f_c(a, b):
    return a + b


@pipefunc(output_name="d")
def f_d(b, c, x=1):  # "c" is the output of f_c
    return b * c


@pipefunc(output_name="e")
def f_e(c, d, x=1):  # "d" is the output of f_d
    return c * d * x

We now have three functions `f_c`, `f_d`, and `f_e`, which we can use to build a pipeline. Let's create a `Pipeline` object, passing our functions in the order we want them to execute. We can also enable debugging, profiling, and caching for the entire pipeline:


In [ ]:
pipeline = Pipeline(
    [f_c, f_d, f_e],
    debug=True,  # optionally print debug information
    profile=True,  # optionally profile the pipeline
    cache_type="lru",  # optionally cache the pipeline
)

Now, we have a pipeline that adds two numbers (function `f_c`), multiplies two numbers (function `f_d`), and again multiplies two numbers (function `f_e`).

---

## Visualizing the Pipeline

You can visualize your pipeline using the `visualize()` method, and print the nodes in the graph using the `graph.nodes` attribute.


In [ ]:
pipeline.visualize()
print("Graph nodes:", pipeline.graph.nodes)

In [ ]:
pipeline.visualize_holoviews()
print("Graph nodes:", pipeline.graph.nodes)

---

## Using the Pipeline

There are several ways in which you can use the pipeline.

### 1. Directly calling the `pipeline` object

If the pipeline has a unique leaf node (single final output), then we can directly call the pipeline object with the input arguments.


In [ ]:
pipeline(a=1, b=2)

In [ ]:
# The above returns the output for:
pipeline.unique_leaf_node

We can also specify the desired output as the first argument of the pipeline function:


In [ ]:
print("`e` is:", pipeline("e", a=1, b=2))
print("`d` is:", pipeline("d", a=1, b=2))

### 3. Using `pipeline.run`

Similar to calling the `pipeline` object, we can use the `run` method to execute the pipeline.

> Note: The `Pipeline.__call__` method is just a wrapper around the `run` method.


In [ ]:
print(pipeline.run(output_name="e", kwargs={"a": 1, "b": 2}))

or get _*all*_ function outputs and inputs by specifying `full_output=True`:


In [ ]:
print(pipeline.run(output_name="e", kwargs={"a": 1, "b": 2}, full_output=True))

### 3. Get a function handle for a specific output (`pipeline.func`)

We can get a handle for each function using the `func` method on the pipeline, passing the output name of the function we want.


In [ ]:
pf_d = pipeline.func("d")
pf_e = pipeline.func("e")

We can now use these handles as if they were the original functions. The pipeline will automatically ensure that the functions are called in the correct order, passing the output of one function as the input to the next.


In [ ]:
c = f_c(a=2, b=3)
assert c == 5
assert (
    f_d(b=3, c=5)
    == pf_d(a=2, b=3)  # We can call pf_d with different arguments
    == pf_d(b=3, c=5)
    == 15
)
assert pf_e(c=c, d=15, x=1) == pf_e(a=2, b=3, x=1) == pf_e(a=2, b=3, d=15, x=1) == 75

The functions returned by `pipeline.func` have several additional methods

#### Using the call_full_output Method

The `call_full_output()` method can be used to call the function and get all the outputs from the pipeline as a dictionary.


In [ ]:
pf_e = pipeline.func("e")
pf_e.call_full_output(a=2, b=3, x=1)

#### Direct Calling with Root Arguments (as positional arguments)

You can directly call the functions in the pipeline with the root arguments using the `call_with_root_args()` method. It automatically executes all the dependencies of the function in the pipeline with the given root arguments.


In [ ]:
pf_e = pipeline.func("e")
pf_e.call_with_root_args(1, 2, 1)  # note these are now positional args

This executes the function `f_e` with the root arguments `a=1, b=2, x=1`.

For more information about this method, you can use the Python built-in `help` function or the `?` command.


In [ ]:
help(pf_e.call_with_root_args)

This shows the signature and the doc-string of the `call_with_root_args` method.


---

## Function Argument Combinations

As seen above, we can call the functions in the pipeline by either providing the root inputs or by providing the output of the previous function ourselves.

To see all the possible combinations of arguments that can be passed to each function, you can use the `all_arg_combinations` property. This will return a dictionary, with function output names as keys and sets of argument tuples as values.


In [ ]:
all_args = pipeline.all_arg_combinations
assert all_args == {
    # means we can call `pipeline("c", a=1, b=2)`
    "c": {("a", "b")},
    # means we can call `pipeline("d", a=1, b=2, x=3)` or `pipeline("d", b=2, c=3, x=4)`
    "d": {("a", "b", "x"), ("b", "c", "x")},
    # means we can call `pipeline("e", a=1, b=2, x=3)` or `pipeline("e", b=2, d=3, x=4)`, etc.
    "e": {("a", "b", "x"), ("a", "b", "d", "x"), ("b", "c", "x"), ("c", "d", "x")},
}
# We can get root arguments for a specific function
assert pipeline.root_args("e") == ("a", "b", "x")

---

## Handling Multiple Outputs

Functions can return multiple results at once. The `output_name` argument allows you to specify multiple outputs by providing a tuple of strings. By default, this assumes the output is a tuple. However, if the output is a single element selected from a tuple, you can use the `output_picker` argument to specify that.


In [ ]:
from pipefunc import pipefunc, Pipeline


# Define a function add_ab with multiple outputs, 'c' and 'const'.
@pipefunc(output_name=("c", "const"))
def add_ab(a, b):
    return (a + b, 1)


# Define a function mul_bc with multiple outputs, 'd' and 'e',
# where output_picker is used to select the output.
@pipefunc(
    output_name=("d", "e"),
    output_picker=dict.__getitem__,
)
def mul_bc(b, c, x=1):
    return {"d": b * c, "e": x}


# Define a function calc_cde with multiple outputs, 'g' and 'h',
# where output_picker is used to select the output.
@pipefunc(
    output_name=("g", "h"),
    output_picker=getattr,
)
def calc_cde(c, d, e, x):
    from types import SimpleNamespace

    return SimpleNamespace(g=c * d * x, h=c + e)


# Define a function add_gh with a single output 'i'.
@pipefunc(output_name="i")
def add_gh(h, g):
    return h + g


# Create a pipeline with the defined functions and visualize it.
pipeline_multiple = Pipeline([add_ab, mul_bc, calc_cde, add_gh])
pipeline_multiple.visualize()
final_func = pipeline_multiple.func("i")
final_func(a=1, b=2, x=3)

**(Sneak peak of a later section: simplifying the pipeline)**

The pipeline can be simplified by combining `calc_cde` and `add_gh` into a single pipeline.


In [ ]:
simplified_pipeline_multiple = pipeline_multiple.simplified_pipeline("i")
simplified_pipeline_multiple.visualize()

Note that, in the simplified pipeline, the full output of `calc_cde` (i.e., `g, h`) is not available.


In [ ]:
# If the full output of calc_cde (g, h) is needed, we can't use the simplified pipeline.
out_full = pipeline_multiple.func("i").call_full_output(a=1, b=2, x=3)
out_full_red = simplified_pipeline_multiple.func("i").call_full_output(a=1, b=2, x=3)
print(f"Full output of f_e:\n{out_full}")
print(f"Full output of f_e after simplification:\n{out_full_red}")

---

## Using the `renames` Feature

The `renames` attribute in `pipefunc` allows you to rename the inputs and outputs of a function before passing them to the next step in the pipeline. This can be particularly useful when the same function is used multiple times in a pipeline, or when you want to provide more meaningful names to the outputs.

### Example: Renaming Outputs

In this example, we demonstrate how to use the `renames` attribute to rename the inputs of a function before they are passed to the next step in the pipeline.

> ⚠️ Instead of using the `@pipefunc` decorator (which creates `pipefunc.PipeFunc` object), we will create `PipeFunc` objects directly and specify the `renames` attribute.


In [ ]:
from pipefunc import Pipeline, PipeFunc


def prod(a, b):
    return a * b


def subtract(a, b):
    return a - b


# We're going to use these functions multiple times in the pipeline
pipeline_renames = Pipeline(
    [
        PipeFunc(prod, output_name="prod1"),
        PipeFunc(prod, output_name="prod2", renames={"a": "x", "b": "y"}),
        PipeFunc(subtract, output_name="delta1", renames={"a": "prod1", "b": "prod2"}),
        PipeFunc(subtract, output_name="delta2", renames={"a": "prod2", "b": "prod1"}),
        PipeFunc(prod, output_name="result", renames={"a": "delta1", "b": "delta2"}),
    ]
)

inputs = {"a": 1, "b": 2, "x": 3, "y": 4}
results = pipeline_renames("result", **inputs)

# Output the results
print("Results:", results)

pipeline_renames.visualize()

#### Explanation

1. **Function Definitions**:

   - `prod(a, b)`: Multiples two numbers and returns the result.
   - `subtract(a, b)`: Subtracts `b` from `a` and returns the result.

2. **Pipeline Construction**:

   - **Step 1**: `PipeFunc(prod, output_name="prod1")`
     - Multiples inputs `a` and `b` and stores the result as `prod1`.
   - **Step 2**: `PipeFunc(prod, output_name="prod2", renames={"a": "x", "b": 'y'})`
     - Multiples inputs `x` and `y` and stores the result as `prod2`.
   - **Step 3**: `PipeFunc(subtract, output_name="delta1", renames={"a": "prod1", "b": "prod2"})`
     - Subtracts `prod2` from `prod1` and stores the result as `delta1`.
   - **Step 4**: `PipeFunc(subtract, output_name="delta2", renames={"a": "prod2", "b": "prod1"})`
     - Subtracts `prod1` from `prod2` and stores the result as `delta2`.
   - **Step 5**: `PipeFunc(prod, output_name="result", renames={"a": "delta1", "b": "delta2"})`
     - Multiples `delta1` and `delta2` and stores the result as `result`.

3. **Inputs and Execution**:

   - The inputs `{"a": 1, "b": 2, "x": 3, "y": 4}` are provided to the pipeline.
   - The pipeline executes in the defined order, renaming inputs and outputs as specified.
   - The final result is computed and returned as `result`.

4. **Output**:
   - The final output of the pipeline is printed, showing the computed `result`.

By using the `renames` attribute, this example demonstrates how to manage and distinguish between different uses of the same function within a pipeline, ensuring the correct values are processed at each step.

---

## Simplifying Pipelines

Consider the following pipeline (look at the `visualize()` output to see the structure of the pipeline):


In [ ]:
from pipefunc import Pipeline


def f1(a, b, c, d):
    return a + b + c + d


def f2(a, b, e):
    return a + b + e


def f3(a, b, f1):
    return a + b + f1


def f4(f1, f3):
    return f1 + f3


def f5(f1, f4):
    return f1 + f4


def f6(b, f5):
    return b + f5


def f7(a, f2, f6):
    return a + f2 + f6


# If the functions are not decorated with @pipefunc,
# they will be wrapped and the output_name will be the function name
pipeline_complex = Pipeline([f1, f2, f3, f4, f5, f6, f7])
pipeline_complex("f7", a=1, b=2, c=3, d=4, e=5)
pipeline_complex.visualize(color_combinable=True)  # combinable functions have the same color

In the example code above, the complex pipeline composed of multiple functions (`f1`, `f2`, `f3`, `f4`, `f5`, `f6`, `f7`) can be simplified by merging the nodes `f1`, `f3`, `f4`, `f5`, `f6` into a single node.
This merging process simplifies the pipeline and allows to reduce the number of functions that need to be cached/saved.

The method `reduced_pipeline` from the `Pipeline` class is used to generate this simplified version of the pipeline.


In [ ]:
simplified_pipeline_complex = pipeline_complex.simplified_pipeline("f7")
simplified_pipeline_complex.visualize()

However, simplifying a pipeline comes with a trade-off. The simplification process removes intermediate nodes that may be necessary for debugging or inspection.

For instance, if a developer wants to monitor the output of `f3` while processing the pipeline, they would not be able to do so in the simplified pipeline as `f3` has been merged into a single node. Hence, while a simplified pipeline can speed up the computation, it may limit the ability to examine intermediate computations.


### Another graph simplification example


In [ ]:
from pipefunc import pipefunc, Pipeline


@pipefunc(output_name=("d", "e"))
def calc_de(b, g, x=1):
    pass


@pipefunc(output_name=("g", "h"))
def calc_gh(a, x=1):
    pass


@pipefunc(output_name="gg")
def calc_gg(g):
    pass


@pipefunc(output_name="i")
def calc_i(gg, b, e):
    pass


# Create a pipeline with the defined functions and visualize it
pipe3 = Pipeline([calc_de, calc_gh, calc_i, calc_gg])
pipe3.visualize(color_combinable=True)
pipe3.simplified_pipeline("i").visualize()

---

## Working with Resources Report

The `resources_report()` method of the `pipeline` provides useful information on the performance of the functions in the pipeline such as CPU usage, memory usage, average time, and the number of times each function was called. This feature is only available if `profile=True` when creating the pipeline.


In [ ]:
# This will print the number of times each function was called
# CPU, memory, and time usage is also reported
pipeline.resources_report()

This report can be beneficial in performance tuning and identifying bottlenecks in your pipeline. You can identify which functions are consuming the most resources and adjust your pipeline accordingly.

You can also look all the stats directly with:


In [ ]:
pipeline.profiling_stats

---

## Parallel Execution and Caching

To enable parallel execution, you can use Python's built-in `concurrent.futures.ProcessPoolExecutor`. To enable caching, simply set the `cache` attribute to `True` for each function. This can be useful to avoid recomputing results when calling the same function with the same arguments multiple times.


In [ ]:
from concurrent.futures import ProcessPoolExecutor

for f in pipeline.functions:
    # Enable caching for all functions
    # See next section to only cache based on a certain parameter sweep
    f.cache = True

pf_e = pipeline.func("e")
with ProcessPoolExecutor(max_workers=1) as executor:
    results = executor.map(pf_e.call_with_dict, [{"a": 2, "b": 3, "x": 1}] * 10)
    print(list(results))

The cache is populated _**even when using parallel execution**_. To see the cache, you can use the `cache` attribute on the pipeline.

The keys of the cache are always in terms of the root arguments.


In [ ]:
print(f"Cache object: {pipeline.cache}")
pipeline.cache.cache

---

## Parameter Sweeps

Parameter sweeps are a technique used in computational simulations to explore the parameter space of a model or system.

In the provided example, the `generate_sweep` method is used to generate a set of combinations of input parameters `a`, `b`, `c`, `d`, and `e` for the function `f7`.
The `generate_sweep` method takes a dictionary of parameters as input and returns a list of dictionaries, where each dictionary represents a combination of parameters.


In [ ]:
from pipefunc.sweep import Sweep

combos = {
    "a": [0, 1, 2],
    "b": [0, 1, 2],
    "c": [0, 1, 2],
    "d": [0, 1, 2],
    "e": [0, 1, 2],
}
# This means a Cartesian product of all the values in the lists
# while zipping ("a", "b").
sweep = Sweep(combos, dims=[("a", "b"), "c", "d", "e"])
sweep.list()[:10]  # show the first 10 combinations

The function `set_cache_for_sweep` then enables caching for nodes in the pipeline that are expected to be executed two or more times during the parameter sweep. Caching improves the efficiency of the sweep by storing and reusing results of repeated computations, rather than performing the same computation multiple times.


In [ ]:
from pipefunc.sweep import set_cache_for_sweep

set_cache_for_sweep("f7", simplified_pipeline_complex, sweep, min_executions=2, verbose=True)

Large parameter sweeps can be computationally expensive. Reducing the pipeline and utilizing caching, as demonstrated in the example, can help alleviate this cost.

---

## Calculating Optimal Execution Order

In complex pipelines, especially those involving parameter sweeps, some function nodes may be executed multiple times.
Precalculating and caching the results of such functions can significantly speed up the pipeline execution.
The `pipefunc` package provides the `get_precalculation_order()` function to determine the optimal execution order of functions in a pipeline, prioritizing those functions which are executed more often.

Let's consider a test scenario with a pipeline composed of four functions and a sweep of parameters `x`, `y`, and `z`.


In [ ]:
from pipefunc import pipefunc, Pipeline
from pipefunc.sweep import Sweep, count_sweep, get_precalculation_order


@pipefunc(output_name=("a", "b"))
def f_ab(y, z=1):
    print(f"🏃 Running f_ab(y={y}, z={z})")
    return y + z, y * z


@pipefunc(output_name=("c", "d"))
def f_cd(x, a, z=1):
    return x + a + z, x * a * z


@pipefunc(output_name="aa")
def f_aa(a):
    print(f"🏃 Running f_aa(a={a})")
    return a + 1


@pipefunc(output_name="i")
def f_i(aa, x, d):
    return aa + x + d


pipeline_order = Pipeline([f_cd, f_ab, f_i, f_aa])
pipeline_order.visualize()

In [ ]:
sweep = Sweep({"y": [1, 2], "x": [3, 4], "z": [5, 6]})
cnt = count_sweep("i", sweep, pipeline_order)
print("Number of executions (keys are based on root_args):")
cnt

To determine the optimal order of execution for functions in the pipeline, we call the `get_precalculation_order()` function, passing in our pipeline and the counts of function executions:


In [ ]:
precalculation_order = get_precalculation_order(pipeline_order, cnt)
precalculation_order

In the `get_precalculation_order()` function, the order is determined by the topological dependencies of the functions and the count of their executions in the context of a parameter sweep. Only functions that are executed multiple times are included in the precalculation order. This ensures that the computation is most efficient where it matters the most.

In this test scenario, the order of precalculation is `[f_e, f_gg]`, meaning `f_e` and `f_gg` should be precalculated and cached before executing the rest of the pipeline.


In [ ]:
print(f"Length of the sweep is {len(sweep)}")
for f in precalculation_order:
    f.cache = True  # enable caching
    func = pipeline_order.func(f.output_name)
    input_args = pipeline_order.root_args(f.output_name)
    sub_sweep = sweep.filtered_sweep(input_args)
    print(f"- Function `{f}` has {len(sub_sweep)} combinations")
    for combo in sub_sweep.generate():  # sweep as generator
        _ = func.call_with_dict(combo)  # We just populate the cache

Now we populated the cache, however, one might also run these calculations on a cluster and save the results to disk.

Note that when we are executing the pipeline to get `'i'`, we are not executing `f_ab` and `f_aa` again, but rather loading the results from the cache.


In [ ]:
# Note no print statements are shown from `f_ab` and `f_aa`
import pandas as pd

F = pipeline_order.func("i")
results = [F.call_full_output(**combo) for combo in sweep.generate()]
df = pd.DataFrame(results)
df

---

## Running Map-Reduce Pipelines using `MapSpec`

This section shows how `pipefunc` uses `mapspec` to automate data distribution across Map-Reduce tasks.

> **Note:** ⚠️ The mapping computation of the pipeline is done in parallel using the `concurrent.futures.ProcessPoolExecutor` whenever `pipeline.map(..., parallel=True)` (default).

### Example: Simple reduction pipeline

The script below demonstrates a two-step pipeline: doubling each integer in an input list, followed by summing all the doubled values.


In [ ]:
from pipefunc import pipefunc, Pipeline
from pipefunc.map import run, load_outputs
import numpy as np
from typing import Any


@pipefunc(output_name="y", mapspec="x[i] -> y[i]")
def double_it(x: int) -> int:
    assert isinstance(x, int)
    return 2 * x


@pipefunc(output_name="sum")
def take_sum(y: np.ndarray[Any, np.dtype[np.int_]]) -> int:
    assert isinstance(y, np.ndarray)
    return sum(y)


pipeline = Pipeline([double_it, take_sum])

inputs = {"x": [0, 1, 2, 3]}
run_folder = "my_run_folder"
results = run(pipeline, inputs, run_folder=run_folder)  # or use `pipeline.map`
assert results["sum"].output == 12

# Or load the outputs from a previous run
assert load_outputs("sum", run_folder=run_folder) == 12
assert load_outputs("y", run_folder=run_folder).tolist() == [0, 2, 4, 6]

#### Explanation

1. **Using `mapspec` for Data Distribution**:

   - The `mapspec` attribute in the `@pipefunc` decorator defines how data is distributed across computations. In `double_it`, `mapspec="x[i] -> y[i]"` specifies that each element `i` of the input array `x` is independently processed to produce the corresponding element `i` in the output array `y`. Because `take_sum` does not have a `mapspec`, it receives the entire array `y` for aggregation.

2. **Function Definitions**:

   - `double_it`: Doubles each integer, demonstrating a simple stateless operation that can be easily parallelized.
   - `take_sum`: Aggregates all elements of the resulting array into a single sum, serving as the reduce step in this Map-Reduce example.

3. **Pipeline Execution**:

   - The `Pipeline.map` method executes the pipeline with specific inputs and a directory for temporary run files, showcasing how PipeFunc manages data flow and execution state across the pipeline. Alternatively, use the `pipefunc.map.run` function.

4. **Results Verification**:

   - Assertions check that the final sum of doubled numbers is correct, ensuring both the integrity and correctness of the pipeline's execution.

5. **Output Retrieval**:
   - The `load_outputs` function demonstrates how to retrieve and verify results post-computation, confirming the output is stored and accessible as expected.

### Example: Cross-product of inputs and multidiagonal aggregation

This example shows how to compute the outer product of two input vectors (`x` and `y`) and then aggregate the resulting matrix along rows, and finally reduce the computation to a single `float` by taking the `norm` of the resulting `aggregated` vector.


In [ ]:
from pipefunc import Pipeline, pipefunc
import numpy as np


@pipefunc(output_name="z", mapspec="x[i], y[j] -> z[i, j]")
def multiply_elements(x: int, y: int) -> int:
    """Multiply two integers."""
    return x * y


@pipefunc(output_name="aggregated", mapspec="z[i, :] -> aggregated[i]")
def aggregate_rows(z: np.ndarray) -> np.ndarray:
    """Sum the elements of each row in matrix z."""
    return np.sum(z)


@pipefunc(output_name="norm")
def compute_norm(aggregated: np.ndarray) -> float:
    """Compute the Euclidean norm of the vector aggregated."""
    return np.linalg.norm(aggregated)


pipeline = Pipeline([multiply_elements, aggregate_rows, compute_norm])
inputs = {"x": [1, 2, 3], "y": [4, 5, 6]}
results = pipeline.map(inputs, run_folder="my_run_folder")  # same as `pipeline.map.run()`
print("Norm of the aggregated sums:", results["norm"].output)

#### Explanation

1. **Matrix Creation (`multiply_elements`)**:

   - Each combination of elements from arrays `x` and `y` is multiplied to form the matrix `z`. The `mapspec` `"x[i], y[j] -> z[i, j]"` ensures that every pair of elements is processed to generate a 2D matrix.

2. **Row Aggregation (`aggregate_rows`)**:

   - The matrix `z` is then processed row by row to sum the values, creating an aggregated result for each row. The `mapspec` `"z[i, :] -> aggregated[i]"` directs the pipeline to apply the summation across each row, transforming a 2D array into a 1D array of row sums.

3. **Vector Norm Calculation (`compute_norm`)**:
   - Finally, the norm of the aggregated vector is computed, providing a single scalar value that quantifies the magnitude of the vector formed from row sums. This step does not require a `mapspec` as it takes the entire output from the previous step and produces a single output.

### Example: Handling Dynamic Output Shapes

It is not always possible to predict output shapes directly from the input parameters. For instance, when a function generates a list whose length depends on the runtime value of an input, PipeFunc cannot automatically determine the output dimensions. To manage such cases, developers must manually specify these output shapes.

We also show an alternative way of specifying the `mapspec` when creating the `Pipeline` object by passing `tuple`s to the `Pipeline` constructor.


In [ ]:
from pipefunc import Pipeline, pipefunc


@pipefunc(output_name="x")
def generate_ints(n: int) -> list[int]:
    """Generate a list of integers from 0 to n-1."""
    return list(range(n))


@pipefunc(output_name="y")
def double_it(x: int) -> int:
    """Double the input integer."""
    return 2 * x


@pipefunc(output_name="sum")
def take_sum(y: list[int]) -> int:
    """Sum a list of integers."""
    return sum(y)


# One is able to specify the mapspec when creating the Pipeline object
# by passing a tuple of the function and the mapspec string.
pipeline = Pipeline(
    [
        generate_ints,
        (double_it, "x[i] -> y[i]"),  # Apply doubling to each element in the list
        take_sum,
    ]
)


inputs = {"n": 4}
internal_shapes = {"x": (4,)}  # Explicitly specify the shape of 'x' output from generate_ints

# Run the pipeline
results = pipeline.map(inputs, internal_shapes=internal_shapes, run_folder="my_run_folder")
print("Sum of doubled integers:", results["sum"].output)

This example demonstrates explicitly specifying output shapes via `internal_shapes` in scenarios where the output size is dynamically determined by input values.

### Example: Zipped Inputs and Pairwise Computation

This pipeline processes zipped inputs `x` and `y` with independent `z` to compute a function across all combinations, producing a 2D matrix `r`.


In [ ]:
from pipefunc import Pipeline, pipefunc


@pipefunc(output_name="r")
def process_elements(x: int, y: int, z: int) -> float:
    return x * y + z


pipeline = Pipeline([(process_elements, "x[a], y[a], z[b] -> r[a, b]")])

inputs = {"x": [1, 2, 3], "y": [4, 5, 6], "z": [7, 8]}

results = pipeline.map(inputs, run_folder="my_run_folder")
output_matrix = results["r"].output
print("Output Matrix:\n", output_matrix)

#### Explanation

- **Function `process_elements`**:

  - Takes three inputs: `x`, `y`, and `z`. For each pair `(x[a], y[a])`, the function is applied with each `z[b]`.

- **Pipeline Definition**:

  - The `mapspec` `"x[a], y[a], z[b] -> r[a, b]"` specifies how elements from the inputs are to be combined. It states that each element from the paired inputs `x` and `y` (indexed by `a`) should be processed with each element from `z` (indexed by `b`), resulting in a 2D output array `r`.

- **Outputs**:
  - The output `r` is a 2-dimensional matrix where the dimensions are determined by the lengths of `x`/`y` and `z`. Each element of this matrix represents the computation result for a specific combination of inputs.


### Example: Physics based example

This example demonstrates using the `pipefunc` for a physics-based simulation. The goal is to create a pipeline for geometry creation, meshing, material assignment, and electrostatics calculations, culminating in computing the average charge.

> Note: this example is based on the [`aiida-dynamic-workflows` tutorial](https://github.com/microsoft/aiida-dynamic-workflows/blob/4d452ed3be4192dc5b2c8f40690f82c3afcaa7a8/examples/02-workflows.md).

We start with defining a few `dataclasses` to represent the geometry, mesh, and material properties. We then define functions to create the geometry, mesh it, assign materials, and calculate the electrostatics.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass


@dataclass(frozen=True)
class Geometry:
    x: float
    y: float


@dataclass(frozen=True)
class Mesh:
    geometry: Geometry
    mesh_size: float


@dataclass(frozen=True)
class Materials:
    geometry: Geometry
    materials: list[str]


@dataclass(frozen=True)
class Electrostatics:
    mesh: Mesh
    materials: Materials
    voltages: list[float]

In [ ]:
import numpy as np

from pipefunc import Pipeline, pipefunc
from pipefunc.map import load_outputs


@pipefunc(output_name="geo")
def make_geometry(x: float, y: float) -> Geometry:
    return Geometry(x, y)


@pipefunc(output_name=("mesh", "coarse_mesh"))
def make_mesh(
    geo: Geometry,
    mesh_size: float,
    coarse_mesh_size: float,
) -> tuple[Mesh, Mesh]:
    return Mesh(geo, mesh_size), Mesh(geo, coarse_mesh_size)


@pipefunc(output_name="materials")
def make_materials(geo: Geometry) -> Materials:
    return Materials(geo, ["i", "j", "c"])


@pipefunc(output_name="electrostatics")
def run_electrostatics(
    mesh: Mesh,
    materials: Materials,
    V_left: float,  # noqa: N803
    V_right: float,  # noqa: N803
) -> Electrostatics:
    return Electrostatics(mesh, materials, [V_left, V_right])


@pipefunc(output_name="charge")
def get_charge(electrostatics: Electrostatics) -> float:
    # obviously not actually the charge; but we should return _some_ number that
    # is "derived" from the electrostatics.
    return sum(electrostatics.voltages)


@pipefunc(output_name="average_charge")
def average_charge(charge: np.ndarray) -> float:
    return np.mean(charge)


pipeline = Pipeline(
    [
        make_geometry,
        make_mesh,
        make_materials,
        (run_electrostatics, "V_left[i], V_right[j] -> electrostatics[i, j]"),
        (get_charge, "electrostatics[i, j] -> charge[i, j]"),
        average_charge,  # this function receives the full 2D array of charges!
    ]
)
pipeline.visualize(figsize=(15, 15))

Let's run the map for some inputs:


In [ ]:
inputs = {
    "V_left": np.linspace(0, 2, 3),
    "V_right": np.linspace(-0.5, 0.5, 2),
    "x": 0.1,
    "y": 0.2,
    "mesh_size": 0.01,
    "coarse_mesh_size": 0.05,
}

run_folder = "my_run_folder"
results = pipeline.map(inputs, run_folder=run_folder, parallel=False)
assert results["average_charge"].output == 1.0
assert results["average_charge"].output_name == "average_charge"
assert load_outputs("average_charge", run_folder=run_folder) == 1.0

This example highlighted how to run a simulation that included a map-reduce operation.
Often we want to sweep this over multiple parameters.
You could add all the `mapspec`s required to map an additional parameter.
Alternatively, you can use the `pipeline.add_mapspec_axis` method to add an axis to parameters of the pipeline.

See the example below, where we extend the `mapspec`s.


In [ ]:
# Add a cross-product of x and y
pipeline.add_mapspec_axis("x", axis="a")
pipeline.add_mapspec_axis("y", axis="b")

# And also a cross-product of the zipped mesh_size and coarse_mesh_size
pipeline.add_mapspec_axis("mesh_size", axis="c")
pipeline.add_mapspec_axis("coarse_mesh_size", axis="c")

# Finally, the mapspecs become, which shows a 3D array for the `average_charge`:
pipeline.mapspecs_as_strings()

In [ ]:
pipeline.visualize(figsize=(20, 20))

Let's run it on a 2x2x2 grid of inputs:


In [ ]:
inputs = {
    "V_left": np.linspace(0, 2, 3),
    "V_right": np.linspace(-0.5, 0.5, 2),
    "x": np.linspace(0.1, 0.2, 2),
    "y": np.linspace(0.2, 0.3, 2),
    "mesh_size": [0.01, 0.02],
    "coarse_mesh_size": [0.05, 0.06],
}
results = pipeline.map(inputs, run_folder=run_folder, parallel=False)
output = results["average_charge"].output
print(f"Output shape: {output.shape}")
print(f"Output:\n{output}")

We can also load all data as `xarray.Dataset`:


In [ ]:
from pipefunc.map import load_xarray_dataset

ds = load_xarray_dataset(run_folder=run_folder)
ds

Or specify the `output_name` to load only specific outputs:


In [ ]:
avg_charge = load_xarray_dataset("average_charge", run_folder=run_folder)
avg_charge

### Custom parallelism

By default when `pipeline.map(..., parallel=True)` is used, the pipeline is executed in parallel using the `concurrent.futures.ProcessPoolExecutor`. However, you can also specify a custom executor to control the parallelism of the pipeline execution.

It works with any custom executor that has the `concurrent.futures.Executor` interface, so for example it works with:

- `concurrent.futures.ProcessPoolExecutor`
- `concurrent.futures.ThreadPoolExecutor`
- `ipyparallel.Client().executor()`
- `dask.distributed.Client().get_executor()`
- `mpi4py.futures.MPIPoolExecutor()`
- `loky.get_reusable_executor()`

To just change the number of cores while using the default executor, use

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import datetime
import numpy as np
import time
from pipefunc import Pipeline, pipefunc


@pipefunc(output_name="double", mapspec="x[i] -> double[i]")
def double_it(x: int) -> int:
    print(f"{datetime.datetime.now()} - Running double_it for x={x}")
    time.sleep(1)
    return 2 * x


@pipefunc(output_name="half", mapspec="x[i] -> half[i]")
def half_it(x: int) -> int:
    print(f"{datetime.datetime.now()} - Running half_it for x={x}")
    time.sleep(1)
    return x // 2


@pipefunc(output_name="sum")
def take_sum(half: np.ndarray, double: np.ndarray) -> int:
    print(f"{datetime.datetime.now()} - Running take_sum")
    return sum(half + double)


pipeline = Pipeline([double_it, half_it, take_sum])
inputs = {"x": [0, 1, 2, 3]}
run_folder = "my_run_folder"
executor = ProcessPoolExecutor(max_workers=8)  # use 8 processes
results = pipeline.map(
    inputs,
    run_folder=run_folder,
    parallel=True,
    executor=executor,
    storage="shared_memory_dict",
)
print(results["sum"].output)

> ⚠️ In this pipeline, `double_it` and `half_it` are doubly parallel; both the map is parallel and the two functions are executed at the same time, note the timestamps and the `sleep()` calls.
> See the `visualize()` output to see the structure of the pipeline.

In [ ]:
pipeline.visualize()